# Notebook 1: Introduction to Monte Carlo Simulation

**Learning Objectives:**
- Understand what Monte Carlo simulation is and why we use it
- Generate random numbers with NumPy
- Estimate pi using random sampling
- Observe how more samples lead to better estimates

---

## What is Monte Carlo Simulation?

Monte Carlo simulation is a technique for solving problems that are too complex to solve analytically. Instead of deriving a formula, we:

1. **Generate many random scenarios**
2. **Apply our logic to each scenario**
3. **Aggregate the results**

The name comes from the Monte Carlo Casino in Monaco - we're essentially "rolling dice" thousands of times to understand the distribution of outcomes.

In [ ]:
# Setup: Import the libraries we'll use
import numpy as np
import matplotlib.pyplot as plt

# Set a random seed for reproducibility
np.random.seed(42)

print("Libraries loaded successfully!")

## Part 1: Generating Random Numbers

NumPy provides several functions for generating random numbers. Let's explore the most common ones.

In [ ]:
# Uniform random numbers between 0 and 1
uniform_samples = np.random.random(10)
print("Uniform [0,1):")
print(uniform_samples)
print()

In [ ]:
# Uniform random numbers in a custom range
# np.random.uniform(low, high, size)
custom_uniform = np.random.uniform(-1, 1, 10)
print("Uniform [-1, 1):")
print(custom_uniform)

In [ ]:
# Standard normal (mean=0, std=1)
normal_samples = np.random.standard_normal(10)
print("Standard Normal:")
print(normal_samples)

In [ ]:
# Normal with custom mean and standard deviation
# np.random.normal(mean, std, size)
custom_normal = np.random.normal(100, 15, 10)  # IQ-like distribution
print("Normal (mean=100, std=15):")
print(custom_normal)

### Exercise 1.1

Generate 1000 samples from a normal distribution with mean=50000 and std=10000 (like annual salaries). Calculate the mean and standard deviation of your sample. How close are they to the true values?

In [ ]:
# YOUR CODE HERE
salaries = np.random.normal(50000, 10000, 1000)

print(f"Sample mean: ${np.mean(salaries):,.2f}")
print(f"Sample std:  ${np.std(salaries):,.2f}")
print(f"True mean:   $50,000.00")
print(f"True std:    $10,000.00")

---

## Part 2: Estimating Pi with Monte Carlo

This is the classic Monte Carlo example. We'll estimate the value of pi by throwing random "darts" at a square and counting how many land inside an inscribed circle.

**The Setup:**
- Draw a square with corners at (-1, -1) and (1, 1)
- Inscribe a circle with radius 1 centered at the origin
- Area of square = 4
- Area of circle = pi * r^2 = pi
- Ratio of areas = pi / 4

If we throw random darts uniformly at the square, the fraction that land inside the circle should approximate pi/4.

In [ ]:
def estimate_pi(n_samples):
    """
    Estimate pi using Monte Carlo simulation.
    
    Args:
        n_samples: Number of random points to generate
    
    Returns:
        Estimate of pi
    """
    # Generate random (x, y) points in [-1, 1] x [-1, 1]
    x = np.random.uniform(-1, 1, n_samples)
    y = np.random.uniform(-1, 1, n_samples)
    
    # Check which points are inside the unit circle
    # A point is inside if x^2 + y^2 <= 1
    inside_circle = (x**2 + y**2) <= 1
    
    # Count points inside
    n_inside = np.sum(inside_circle)
    
    # Estimate pi
    # (n_inside / n_samples) ≈ pi/4
    # Therefore: pi ≈ 4 * (n_inside / n_samples)
    pi_estimate = 4 * n_inside / n_samples
    
    return pi_estimate

# Test with different sample sizes
for n in [100, 1000, 10000, 100000, 1000000]:
    estimate = estimate_pi(n)
    error = abs(estimate - np.pi)
    print(f"n = {n:>10,}: pi ≈ {estimate:.6f}, error = {error:.6f}")

### Visualizing the Dart Throws

In [ ]:
# Generate points for visualization
n_viz = 1000
x = np.random.uniform(-1, 1, n_viz)
y = np.random.uniform(-1, 1, n_viz)
inside = (x**2 + y**2) <= 1

# Create the plot
fig, ax = plt.subplots(figsize=(8, 8))

# Plot points inside (blue) and outside (red)
ax.scatter(x[inside], y[inside], c='blue', alpha=0.5, s=10, label='Inside circle')
ax.scatter(x[~inside], y[~inside], c='red', alpha=0.5, s=10, label='Outside circle')

# Draw the circle
theta = np.linspace(0, 2*np.pi, 100)
ax.plot(np.cos(theta), np.sin(theta), 'k-', linewidth=2)

# Draw the square
ax.plot([-1, 1, 1, -1, -1], [-1, -1, 1, 1, -1], 'k-', linewidth=2)

ax.set_xlim(-1.1, 1.1)
ax.set_ylim(-1.1, 1.1)
ax.set_aspect('equal')
ax.legend()
ax.set_title(f'Monte Carlo Pi Estimation\n{np.sum(inside)}/{n_viz} inside → pi ≈ {4*np.sum(inside)/n_viz:.4f}')
plt.show()

---

## Part 3: Convergence - How Many Samples Do We Need?

A key question in Monte Carlo: how many samples are enough? Let's track how our estimate improves as we add more samples.

In [ ]:
# Track convergence
max_samples = 50000
x = np.random.uniform(-1, 1, max_samples)
y = np.random.uniform(-1, 1, max_samples)
inside = (x**2 + y**2) <= 1

# Calculate running estimate
cumulative_inside = np.cumsum(inside)
sample_counts = np.arange(1, max_samples + 1)
running_estimate = 4 * cumulative_inside / sample_counts

# Plot convergence
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(sample_counts, running_estimate, 'b-', alpha=0.7, linewidth=0.5)
ax.axhline(y=np.pi, color='r', linestyle='--', label=f'True π = {np.pi:.6f}')
ax.set_xlabel('Number of Samples')
ax.set_ylabel('Estimate of π')
ax.set_title('Convergence of Monte Carlo Pi Estimate')
ax.legend()
ax.set_ylim(2.8, 3.5)
plt.show()

print(f"Final estimate with {max_samples:,} samples: {running_estimate[-1]:.6f}")
print(f"Error: {abs(running_estimate[-1] - np.pi):.6f}")

### The 1/√n Rule

Monte Carlo error decreases proportionally to 1/√n. This means:
- To halve the error, you need 4x the samples
- To reduce error by 10x, you need 100x the samples

This is why we often use 10,000 or 100,000 simulations - it's a balance between accuracy and computation time.

In [ ]:
# Demonstrate the 1/sqrt(n) convergence
n_trials = 100
sample_sizes = [100, 400, 1600, 6400, 25600]

print("Sample Size | Avg Error | Expected Ratio")
print("-" * 45)

prev_error = None
for n in sample_sizes:
    errors = [abs(estimate_pi(n) - np.pi) for _ in range(n_trials)]
    avg_error = np.mean(errors)
    
    if prev_error is not None:
        ratio = prev_error / avg_error
        print(f"{n:>11,} | {avg_error:.6f} | {ratio:.2f}x (expect ~2x)")
    else:
        print(f"{n:>11,} | {avg_error:.6f} | (baseline)")
    
    prev_error = avg_error

---

## Summary

In this notebook, you learned:

1. **Monte Carlo converts hard math into easy counting** - Instead of deriving formulas, we simulate and count outcomes

2. **NumPy provides the random number generators** - `np.random.uniform()`, `np.random.normal()`, etc.

3. **More samples = better estimates** - Error decreases as 1/√n

4. **10,000 samples is often a good starting point** - Balances accuracy with computation time

---

## Next: Notebook 2 - Simulating Stock Prices

We'll apply these same concepts to simulate stock market returns using Geometric Brownian Motion.